---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [19]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [20]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [21]:
def graph_identification():
#     li=list()
#     for G in P1_Graphs:
#         degrees = G.degree()
#         degree_values = sorted(set(degrees.values()))
#         histogram =[list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#         import matplotlib.pyplot as plt
#         plt.bar(degree_values,histogram)
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes')
#         plt.show()
#         plt.plot(degree_values,histogram, 'o')
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes')
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.show()
#         li.append((nx.average_clustering(G),nx.average_shortest_path_length(G)))
         
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()# Your Answer Here

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [22]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [24]:
def salary_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['c_c']=pd.Series(nx.closeness_centrality(G, normalized = True))
    df['b_c']=pd.Series(nx.betweenness_centrality(G,normalized = True, endpoints = False,k=50))
    traindf=df.dropna()
    testdf=df[df.ManagementSalary.isnull()==True].drop(['ManagementSalary'],axis=1)
    clf = GradientBoostingClassifier(random_state = 0).fit(traindf.drop(['ManagementSalary'],axis=1), traindf['ManagementSalary'])
    y_pp=clf.predict_proba(testdf)[:,1]
    s=pd.Series(y_pp,index=testdf.index.values.tolist())
    return s
salary_predictions()# Your Answer Here

1       0.015323
2       0.926676
5       0.986936
8       0.092534
14      0.183873
18      0.057922
27      0.081621
30      0.814010
31      0.206866
34      0.017103
37      0.009975
40      0.083814
45      0.022562
54      0.222955
55      0.693707
60      0.081063
62      0.985576
65      0.856238
77      0.009347
79      0.021294
97      0.013106
101     0.004028
103     0.635745
108     0.030764
113     0.099523
122     0.002162
141     0.477339
142     0.978858
144     0.011132
145     0.459632
          ...   
913     0.497262
914     0.007444
915     0.001975
918     0.029122
923     0.013683
926     0.038319
931     0.007027
934     0.001487
939     0.001825
944     0.001871
945     0.013683
947     0.033178
950     0.168294
951     0.004519
953     0.003668
959     0.001825
962     0.001825
963     0.097280
968     0.027244
969     0.027244
974     0.027850
984     0.001487
987     0.025766
989     0.031367
991     0.027244
992     0.001825
994     0.001753
996     0.0017

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [25]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [31]:
def new_connections_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    future_connections['c_n'] = [len(list(nx.common_neighbors(G,u,v))) for (u,v) in future_connections.index.values.tolist()]
    future_connections['j_c'] = [i[2] for i in nx.jaccard_coefficient(G,future_connections.index)]
    future_connections['r_a_i'] = [i[2] for i in nx.resource_allocation_index(G,future_connections.index)]
    future_connections['p_a'] = [i[2] for i in nx.preferential_attachment(G,future_connections.index)]
    
    traindf=future_connections.dropna()
    testdf=future_connections[future_connections['Future Connection'].isnull()==True].drop(['Future Connection'],axis=1)
    clf = GradientBoostingClassifier(random_state = 0).fit(traindf.drop(['Future Connection'],axis=1),
                                                           traindf['Future Connection'])
    y_pp=clf.predict_proba(testdf)[:,1]
    s=pd.Series(y_pp,index=testdf.index.values.tolist())
    return s
new_connections_predictions()

(107, 348)    0.029567
(542, 751)    0.013001
(20, 426)     0.562763
(50, 989)     0.013001
(942, 986)    0.013001
(324, 857)    0.013001
(13, 710)     0.118918
(19, 271)     0.138913
(319, 878)    0.013001
(659, 707)    0.013001
(49, 843)     0.013001
(208, 893)    0.013001
(377, 469)    0.013475
(405, 999)    0.019043
(129, 740)    0.015585
(292, 618)    0.049497
(239, 689)    0.013001
(359, 373)    0.013475
(53, 523)     0.031658
(276, 984)    0.013001
(202, 997)    0.013001
(604, 619)    0.092708
(270, 911)    0.013001
(261, 481)    0.059519
(200, 450)    0.904397
(213, 634)    0.013001
(644, 735)    0.154224
(346, 553)    0.013001
(521, 738)    0.013001
(422, 953)    0.018690
                ...   
(672, 848)    0.013001
(28, 127)     0.970584
(202, 661)    0.013001
(54, 195)     0.997258
(295, 864)    0.013001
(814, 936)    0.013001
(839, 874)    0.013001
(139, 843)    0.013001
(461, 544)    0.013173
(68, 487)     0.013173
(622, 932)    0.013001
(504, 936)    0.016074
(479, 528) 